# Unit 2 Lesson 7 - Joint 2D and AngryAvians
In this lesson, we'll learn about new Unity physics components called Joint2D and use everything we learned so far to create AngryAvians(Angry bird clone).

---

# Learning Objectives
---

*   Students will learn about Joint2D
*   Students will apply previous lesson to create an Angry Birds clone

# Key Concepts
---

*   Joint2D
*   Inheritance

# Introduction
---

# Concept 1: Joint2D

---
### **Joint2D**

#### What is ? 


#### How to use ?




# Concept 2: Inheritance
---
#### What is Inheritance? 




## Activity


---
## Activity #1

---

**Problem:** 

##### **Solution**


---
## Activity #2

---

**Problem:**

##### **Solution**


 

# HOMEWORK
---

## HW #1

---

**Problem:** 

##### **Solution**

